In [1]:
import os, json, math, gc
import numpy as np
import pandas as pd

from pathlib import Path
import joblib

from sklearn.metrics import roc_auc_score, brier_score_loss, roc_curve
from scipy.stats import chi2

#  PATHS
DATA_DIR    = Path("/content/drive/MyDrive/freddie mac")
FEAT_DIR    = DATA_DIR / "features"
MODELS_DIR  = DATA_DIR / "models"

PD_FEATURES_FILE  = FEAT_DIR / "features_pd.parquet"
META_FILE         = FEAT_DIR / "feature_meta.json"

# Preferred PD model (will gracefully fall back if missing)
PREFERRED_PD_FILES = [
    MODELS_DIR / "pd_model_xgb_isotonic.joblib",    # preferred
    MODELS_DIR / "pd_model_rf_cal.joblib",          # fallback
    MODELS_DIR / "pd_model_logit_calibrated.joblib" # fallback
]

# LGD models + calibrators (origination-only)
LGD_MODELS = {
    "xgb":  MODELS_DIR / "lgd_model_xgb.joblib",
    "rf":   MODELS_DIR / "lgd_model_rf.joblib",
    "ridge":MODELS_DIR / "lgd_model_logit_ols.joblib"  # dict {'ridge':..., 'num_pipe':...}
}
LGD_CAL   = {
    "xgb":   MODELS_DIR / "lgd_calibrator_xgb_orig_only.joblib",
    "rf":    MODELS_DIR / "lgd_calibrator_rf_orig_only.joblib",
    "ridge": MODELS_DIR / "lgd_calibrator_ridge_orig_only.joblib"
}

# EAD/CCF models + calibrators
EAD_MODELS = {
    "xgb":   MODELS_DIR / "ead_model_xgb.joblib",
    "rf":    MODELS_DIR / "ead_model_rf.joblib",
    "ridge": MODELS_DIR / "ead_model_ridge.joblib"  # dict {'ridge':..., 'num_pipe':...}
}
EAD_CAL = {
    "xgb":   MODELS_DIR / "ead_ccf_calibrator_xgb.joblib",
    "rf":    MODELS_DIR / "ead_ccf_calibrator_rf.joblib",
    "ridge": MODELS_DIR / "ead_ccf_calibrator_ridge.joblib"
}

# Monte-Carlo settings
N_DRAWS     = 3000           # portfolio simulations
CHUNK_SIZE  = 120_000        # score/MC in chunks to control memory
RNG_SEED    = 42

# Stress scenario multipliers (EBA-style, simple scalars)
SCENARIOS = {
    "Baseline": {"pd_mult": 1.00, "lgd_mult": 1.00, "ccf_mult": 1.00},
    "Adverse":  {"pd_mult": 1.50, "lgd_mult": 1.10, "ccf_mult": 1.05},
    "Severe":   {"pd_mult": 2.50, "lgd_mult": 1.20, "ccf_mult": 1.10},
}

In [2]:
# Load portfolio feature table (one row per loan)
pd_df = pd.read_parquet(PD_FEATURES_FILE)
print("Rows:", len(pd_df))
print("Columns sample:", pd_df.columns.tolist()[:25])

# Identify feature columns used during training
# numeric + WoE (adjust automatically to what's present)
NUM_CANDS = [
    'credit_score','original_debt_to_income_dti_ratio',
    'original_loan_to_value_ltv','original_combined_loan_to_value_cltv',
    'original_interest_rate','original_upb','orig_upb' # one of these will exist
]
WOE_CANDS = [
    'channel_woe','occupancy_status_woe','property_type_woe',
    'loan_purpose_woe','property_state_woe'
]

X_cols = [c for c in NUM_CANDS+WOE_CANDS if c in pd_df.columns]
# prefer 'original_upb' over 'orig_upb' if both exist
if 'original_upb' in X_cols and 'orig_upb' in X_cols: X_cols.remove('orig_upb')

assert len(X_cols)>0, "No model features found in features_pd.parquet!"

# Determine original UPB column for EAD back-transform
ORIG_UPB_COL = 'original_upb' if 'original_upb' in pd_df.columns else ('orig_upb' if 'orig_upb' in pd_df.columns else None)
assert ORIG_UPB_COL is not None, "Need original_upb/orig_upb in features file."


Rows: 737500
Columns sample: ['loan_sequence_number', 'original_upb', 'credit_score', 'original_debt_to_income_dti_ratio', 'original_loan_to_value_ltv', 'original_combined_loan_to_value_cltv', 'original_interest_rate', 'channel', 'occupancy_status', 'property_type', 'loan_purpose', 'property_state', 'first_payment_date', 'maturity_date', 'pd_default_flag', '__first_90dpd_month', '__first_liq_month', 'dti_bin', 'ltv_bin', 'channel_woe', 'occupancy_status_woe', 'property_type_woe', 'loan_purpose_woe', 'property_state_woe']


In [3]:
def ks_stat(y_true, p):
    fpr, tpr, thr = roc_curve(y_true, p)
    return float((tpr - fpr).max())

def hosmer_lemeshow(y_true, p_hat, bins=10):
    # equal-count bins on predicted probability
    df = pd.DataFrame({"y": y_true.astype(int), "p": p_hat}).sort_values("p")
    df["bin"] = pd.qcut(df["p"], bins, duplicates="drop")
    g = df.groupby("bin").agg(n=("y","size"), obs=("y","sum"), pred=("p","sum")).reset_index(drop=True)
    g["exp"] = g["pred"]
    g["obs_non"] = g["n"] - g["obs"]
    g["exp_non"] = g["n"] - g["exp"]
    # HL statistic
    with np.errstate(divide="ignore", invalid="ignore"):
        part1 = (g["obs"] - g["exp"])**2 / (g["exp"] + 1e-12)
        part2 = (g["obs_non"] - g["exp_non"])**2 / (g["exp_non"] + 1e-12)
    hl = float((part1 + part2).sum())
    dfreedom = 8  # bins-2
    pval = 1 - chi2.cdf(hl, dfreedom)
    return hl, pval, g

def print_pd_metrics(y_true, proba, name="PD"):
    auc = roc_auc_score(y_true, proba)
    ks  = ks_stat(y_true, proba)
    brier = brier_score_loss(y_true, proba)
    hl, pval, _ = hosmer_lemeshow(y_true, proba, bins=10)
    print(f"[{name}] AUC={auc:.4f} | KS={ks:.4f} | HL={hl:,.2f} (p={pval:.3f}) | Brier={brier:.5f}")

def load_first_existing(paths):
    for p in paths:
        if Path(p).exists():
            return joblib.load(p), Path(p).name
    raise FileNotFoundError(f"None of the PD model files exist: {paths}")


In [4]:
#  PD (pick the first available calibrated model, preferring XGB-isotonic)
pd_model, pd_model_name = load_first_existing(PREFERRED_PD_FILES)
print("Loaded PD model:", pd_model_name)

#  LGD (origination-only)
lgd_loaded = {}
lgd_cal    = {}
for k, path in LGD_MODELS.items():
    if path.exists():
        lgd_loaded[k] = joblib.load(path)
for k, path in LGD_CAL.items():
    if path.exists():
        lgd_cal[k] = joblib.load(path)
print("LGD models loaded:", list(lgd_loaded.keys()))
print("LGD calibrators loaded:", list(lgd_cal.keys()))

#  EAD / CCF
ead_loaded = {}
ead_cal    = {}
for k, path in EAD_MODELS.items():
    if path.exists():
        ead_loaded[k] = joblib.load(path)
for k, path in EAD_CAL.items():
    if path.exists():
        ead_cal[k] = joblib.load(path)
print("EAD models loaded:", list(ead_loaded.keys()))
print("EAD calibrators loaded:", list(ead_cal.keys()))


Loaded PD model: pd_model_xgb_isotonic.joblib
LGD models loaded: ['xgb', 'rf', 'ridge']
LGD calibrators loaded: ['xgb', 'rf', 'ridge']
EAD models loaded: ['xgb', 'rf', 'ridge']
EAD calibrators loaded: ['xgb', 'rf', 'ridge']


In [6]:
# Build model input X
X = pd_df[X_cols].copy()
for c in X.columns:
    X[c] = pd.to_numeric(X[c], errors='coerce')


#  Align X columns to exactly what the saved PD model expects
def _unwrap_estimator(est):
    # peel calibrated/grid/pipeline wrappers to reach the actual classifier
    if hasattr(est, "calibrated_classifiers_"):          # CalibratedClassifierCV
        return _unwrap_estimator(est.calibrated_classifiers_[0].estimator)
    if hasattr(est, "best_estimator_"):                  # GridSearchCV
        return _unwrap_estimator(est.best_estimator_)
    if hasattr(est, "steps"):                            # Pipeline
        return _unwrap_estimator(est.steps[-1][1])
    return est

def _get_expected_feature_names(est):
    base = _unwrap_estimator(est)
    # sklearn-compatible
    if hasattr(base, "feature_names_in_"):
        return list(base.feature_names_in_)
    # xgboost booster stores names if fitted on a DataFrame
    try:
        if hasattr(base, "get_booster"):
            bn = base.get_booster().feature_names
            if bn is not None:
                return list(bn)
    except Exception:
        pass
    return None

expected = _get_expected_feature_names(pd_model)
if expected is not None:
    missing = [c for c in expected if c not in X.columns]
    extra   = [c for c in X.columns   if c not in expected]
    if missing:
        raise ValueError(f"Features required by the PD model are missing from your table: {missing}")
    if extra:
        print("Dropping extra columns not used in PD training:", extra)
    # reorder & drop extras to match training exactly
    X = X[expected]




# Predict PD (pd_model is already a pipeline or a calibrated wrapper)
pd_hat = pd_model.predict_proba(X)[:,1]
pd_df['PD'] = np.clip(pd_hat, 1e-6, 1-1e-6)

# If labels exist, re-create the time split and validate
def make_splits(df, date_cols=('first_payment_date','orig_month','__first_default_month'),
                train_end='2018-12-01', valid_end='2021-12-01'):
    date_col = next((c for c in date_cols if c in df.columns), None)
    if date_col is None:
        r = np.random.RandomState(42).rand(len(df))
        return (r<0.6), ((r>=0.6)&(r<0.8)), (r>=0.8), None
    d = pd.to_datetime(df[date_col], errors='coerce')
    train = d <= pd.to_datetime(train_end)
    valid = (d > pd.to_datetime(train_end)) & (d <= pd.to_datetime(valid_end))
    test  = d > pd.to_datetime(valid_end)
    return train.fillna(False), valid.fillna(False), test.fillna(False), date_col

if 'pd_default_flag' in pd_df.columns:
    y = pd_df['pd_default_flag'].astype(int)
    tr, va, te, split_col = make_splits(pd_df)
    print("PD split by:", split_col or "random")
    print_pd_metrics(y[va], pd_df.loc[va,'PD'].values, f"PD ({pd_model_name}) VALID")
    print_pd_metrics(y[te], pd_df.loc[te,'PD'].values, f"PD ({pd_model_name}) TEST ")


Dropping extra columns not used in PD training: ['original_combined_loan_to_value_cltv']
PD split by: first_payment_date
[PD (pd_model_xgb_isotonic.joblib) VALID] AUC=0.7693 | KS=0.4078 | HL=0.00 (p=1.000) | Brier=0.03093
[PD (pd_model_xgb_isotonic.joblib) TEST ] AUC=0.7189 | KS=0.3425 | HL=5,385.93 (p=0.000) | Brier=0.02039


/tmp/ipython-input-1010402527.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = df.groupby("bin").agg(n=("y","size"), obs=("y","sum"), pred=("p","sum")).reset_index(drop=True)
/tmp/ipython-input-1010402527.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = df.groupby("bin").agg(n=("y","size"), obs=("y","sum"), pred=("p","sum")).reset_index(drop=True)


In [7]:
def predict_lgd(model_obj, calibrator, X_df):
    """
    model_obj: either a Pipeline (RF/XGB) or dict {'ridge', 'num_pipe'} for logit-OLS
    calibrator: IsotonicRegression fitted on VALID (maps raw LGD->calibrated LGD)
    """
    if isinstance(model_obj, dict):  # ridge + scaler
        ridge = model_obj['ridge']; num_pipe = model_obj['num_pipe']
        Xp = num_pipe.transform(X_df)
        # model predicts logit(LGD) → inverse-logit
        z = ridge.predict(Xp)
        lgd_raw = 1/(1+np.exp(-z))
    else:
        lgd_raw = model_obj.predict(X_df)
    lgd_raw = np.clip(lgd_raw, 0.0, 1.0)
    if calibrator is not None:
        lgd_cal = calibrator.transform(lgd_raw)
        return np.clip(lgd_cal, 0.0, 1.0)
    return lgd_raw

# choose LGD model to use (default to XGB if present, else RF, else Ridge)
LGD_PICK = next((k for k in ["xgb","rf","ridge"] if k in lgd_loaded), None)
assert LGD_PICK is not None, "No LGD model files found."
lgd_model_obj = lgd_loaded[LGD_PICK]
lgd_calib     = lgd_cal.get(LGD_PICK, None)

pd_df['LGD'] = predict_lgd(lgd_model_obj, lgd_calib, X)
print("LGD model used:", LGD_PICK, "| mean LGD:", pd_df['LGD'].mean())


LGD model used: xgb | mean LGD: 0.96000147


In [9]:

def predict_ccf(model_obj, X_df, calibrator=None):
    """Score CCF with the same preprocessing as training, then (optionally) apply isotonic calibrator."""
    if isinstance(model_obj, dict):  # ridge + scaler bundle
        ridge = model_obj['ridge']; num_pipe = model_obj['num_pipe']
        Xp = num_pipe.transform(X_df)
        ccf_raw = ridge.predict(Xp)
    else:  # Pipeline([impute, model]) for RF / XGB
        ccf_raw = model_obj.predict(X_df)

    ccf_raw = np.clip(ccf_raw, 0.0, 2.0)
    if calibrator is not None:
        ccf_cal = calibrator.transform(ccf_raw)
        return np.clip(ccf_cal, 0.0, 2.0)
    return ccf_raw


def ead_expected_features(model_obj):
    """Return the feature list the fitted EAD model expects (from the imputer/scaler)."""
    # dict bundle: use the fitted preprocessing pipeline
    if isinstance(model_obj, dict) and 'num_pipe' in model_obj:
        pipe = model_obj['num_pipe']
        if hasattr(pipe, 'named_steps'):
            for step in ['impute', 'scale']:
                st = pipe.named_steps.get(step)
                if st is not None and hasattr(st, 'feature_names_in_'):
                    return list(st.feature_names_in_)
        if hasattr(pipe, 'feature_names_in_'):
            return list(pipe.feature_names_in_)

    # sklearn Pipeline for RF/XGB: imputer step holds names
    if hasattr(model_obj, 'named_steps'):
        imp = model_obj.named_steps.get('impute')
        if imp is not None and hasattr(imp, 'feature_names_in_'):
            return list(imp.feature_names_in_)
        if hasattr(model_obj, 'feature_names_in_'):
            return list(model_obj.feature_names_in_)

    # last resort
    return None


# Build the scoring matrix EXACTLY as the model was trained
ead_feat_list = ead_expected_features(ead_loaded.get('xgb') or ead_loaded.get('rf') or ead_loaded.get('ridge'))
if ead_feat_list is None:
    raise RuntimeError("Could not infer EAD model feature list; please keep the imputer/scaler fitted when saving.")

# Make X_ead in the correct order and with the same columns
X_ead = pd_df.reindex(columns=ead_feat_list, fill_value=0.0).copy()
for c in X_ead.columns:
    X_ead[c] = pd.to_numeric(X_ead[c], errors='coerce')

# Pick the best available model in your folder (xgb → rf → ridge), and its calibrator
EAD_PICK = next((k for k in ["xgb","rf","ridge"] if k in ead_loaded), None)
assert EAD_PICK is not None, "No EAD model files found."
ead_model_obj = ead_loaded[EAD_PICK]
ead_calib     = ead_cal.get(EAD_PICK, None)

# Compute CCF and EAD
pd_df['CCF'] = predict_ccf(ead_model_obj, X_ead, calibrator=ead_calib)

# Ensure ORIG_UPB_COL is defined (use the one from earlier cells)
if 'ORIG_UPB_COL' not in globals() or ORIG_UPB_COL not in pd_df.columns:
    ORIG_UPB_COL = next((c for c in ['original_upb','orig_upb'] if c in pd_df.columns), None)
    assert ORIG_UPB_COL is not None, "Missing original UPB column."

pd_df['EAD'] = pd.to_numeric(pd_df[ORIG_UPB_COL], errors='coerce') * pd_df['CCF']
print("EAD model used:", EAD_PICK, "| mean CCF:", pd_df['CCF'].mean())


EAD model used: xgb | mean CCF: 0.9827604


In [11]:
pd_df['EL'] = pd_df['PD'] * pd_df['LGD'] * pd_df['EAD']
print("Portfolio EL ($):", pd_df['EL'].sum().round(2))

def money(x):
    return f"${x:,.0f}"

by_state = pd_df.groupby('property_state', dropna=False)['EL'].sum().sort_values(ascending=False).head(10)
by_vintage = pd_df.assign(vintage=pd.to_datetime(pd_df.get('first_payment_date'), errors='coerce').dt.year)\
                  .groupby('vintage')['EL'].sum().sort_values(ascending=False).head(10)
by_ltv = pd_df.groupby('ltv_bin', dropna=False)['EL'].sum().sort_values(ascending=False)

print("\nTop-10 EL by state:\n", by_state.map(lambda x: float(x)).apply(money))
print("\nTop-10 EL by vintage:\n", by_vintage.map(lambda x: float(x)).apply(money))
print("\nEL by LTV bin:\n", by_ltv.map(lambda x: float(x)).apply(money))


Portfolio EL ($): 7639337165.83

Top-10 EL by state:
 property_state
CA    $1,119,956,268
FL      $843,212,242
TX      $677,187,042
NY      $550,468,209
NJ      $347,280,073
IL      $335,231,567
GA      $249,682,939
MD      $214,074,150
PA      $213,831,529
WA      $190,162,203
Name: EL, dtype: object

Top-10 EL by vintage:
 vintage
2023    $983,445,801
2024    $909,899,377
2022    $712,321,968
2019    $540,404,237
2018    $534,901,346
2017    $522,325,510
2016    $459,351,660
2015    $426,193,269
2014    $425,711,267
2020    $396,514,049
Name: EL, dtype: object

EL by LTV bin:
 ltv_bin
70-80    $2,411,669,265
90-95    $1,652,208,096
80-90    $1,130,574,077
<=60       $882,992,466
60-70      $808,252,876
>95        $753,640,385
Name: EL, dtype: object


/tmp/ipython-input-3367231370.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  by_ltv = pd_df.groupby('ltv_bin', dropna=False)['EL'].sum().sort_values(ascending=False)


In [12]:
rng = np.random.default_rng(RNG_SEED)

# Fixed severities per loan
loss_if_default = (pd_df['LGD'].values * pd_df['EAD'].values).astype('float64')
pdv = pd_df['PD'].values.astype('float64')

def simulate_portfolio_losses(n_draws=N_DRAWS, chunk=CHUNK_SIZE):
    n = len(pdv)
    losses = np.zeros(n_draws, dtype='float64')
    for start in range(0, n, chunk):
        end = min(start+chunk, n)
        p = pdv[start:end]
        sev = loss_if_default[start:end]
        # For memory: simulate in blocks of draws as well
        # We do draws in batches of 250 to keep RAM small
        B = 250
        for d0 in range(0, n_draws, B):
            d1 = min(d0+B, n_draws)
            u = rng.random((d1-d0, end-start))
            default_mat = (u < p)   # boolean
            # sum losses per draw
            losses[d0:d1] += default_mat @ sev
        del u, default_mat
        gc.collect()
    return losses

losses = simulate_portfolio_losses()
def var_cvar(samples, alpha=0.99):
    # VaR = alpha-quantile; CVaR = mean of tail beyond VaR
    q = np.quantile(samples, alpha)
    tail = samples[samples >= q]
    return float(q), float(tail.mean())

for a in (0.95, 0.99):
    v, c = var_cvar(losses, alpha=a)
    print(f"{int(a*100)}% VaR = {money(v)} | {int(a*100)}% CVaR ≈ {money(c)}")


95% VaR = $7,715,388,397 | 95% CVaR ≈ $7,736,647,168
99% VaR = $7,748,897,233 | 99% CVaR ≈ $7,769,372,745


In [13]:
def apply_stress(pd_base, lgd_base, ccf_base, s):
    pd_st  = np.clip(pd_base * s["pd_mult"], 1e-6, 1-1e-6)
    lgd_st = np.clip(lgd_base * s["lgd_mult"], 0.0, 1.0)
    ccf_st = np.clip(ccf_base * s["ccf_mult"], 0.0, 2.0)
    ead_st = ccf_st * pd_df[ORIG_UPB_COL].values
    el_st  = pd_st * lgd_st * ead_st
    return pd_st, lgd_st, ead_st, el_st

for name, s in SCENARIOS.items():
    _, _, _, el = apply_stress(pd_df['PD'].values, pd_df['LGD'].values, pd_df['CCF'].values, s)
    print(f"[{name}] Portfolio EL = {money(el.sum())}")

    # (Optional) simulate portfolio VaR under this scenario (same severities but stressed PD/LGD/EAD)
    pdv_st, lgd_st, ead_st, _ = apply_stress(pd_df['PD'].values, pd_df['LGD'].values, pd_df['CCF'].values, s)
    loss_if_default_st = (lgd_st * ead_st).astype('float64')

    def simulate_stressed(n_draws=N_DRAWS, chunk=CHUNK_SIZE):
        n = len(pdv_st); out = np.zeros(n_draws, dtype='float64')
        for start in range(0, n, chunk):
            end = min(start+chunk, n)
            p = pdv_st[start:end]; sev = loss_if_default_st[start:end]
            B=250
            for d0 in range(0, n_draws, B):
                d1 = min(d0+B, n_draws)
                u = rng.random((d1-d0, end-start))
                out[d0:d1] += (u < p) @ sev
        return out

    losses_st = simulate_stressed()
    for a in (0.95, 0.99):
        v, c = var_cvar(losses_st, alpha=a)
        print(f"   {name} {int(a*100)}% VaR = {money(v)} | CVaR ≈ {money(c)}")


[Baseline] Portfolio EL = $7,639,337,166
   Baseline 95% VaR = $7,714,275,829 | CVaR ≈ $7,732,523,341
   Baseline 99% VaR = $7,740,825,611 | CVaR ≈ $7,760,336,891
[Adverse] Portfolio EL = $12,561,110,585
   Adverse 95% VaR = $12,662,909,372 | CVaR ≈ $12,689,002,506
   Adverse 99% VaR = $12,703,716,770 | CVaR ≈ $12,728,264,438
[Severe] Portfolio EL = $21,851,430,795
   Severe 95% VaR = $21,981,807,981 | CVaR ≈ $22,012,356,903
   Severe 99% VaR = $22,034,259,594 | CVaR ≈ $22,059,698,882
